In [5]:
import pandas as pd

In [69]:
all_questions_res = pd.read_excel('WFH Curation.xlsx',sheet_name=1,usecols=[0,1,2],skiprows=[0])

In [70]:
all_questions_res

,Response,Topic,Subtopic
0,My organisation has slowly and steadily infuse...,Adopted hybrid work policy,Adopted hybrid work policy
1,Not formally. Employees are strongly encourage...,Adopted hybrid work policy,Adopted hybrid work policy
2,"Yes, we are now told to work atleast three da...",Adopted hybrid work policy,Adopted hybrid work policy
3,"Yes. We do not require people to come in, but...",Adopted hybrid work policy,Adopted hybrid work policy
4,"Yes - if not a virtual employee, required to b...",Adopted hybrid work policy,Adopted hybrid work policy
...,...,...,...
171,"yes it has changed, we have been asked to come...",Returned to office,Returned to office
172,"It has changed, and increased the number of da...",Returned to office,Returned to office
173,Yes. We are back in the office.,Returned to office,Returned to office
174,"Yes it has, we have a RTO office policy",Returned to office,Returned to office


In [10]:
topic_sub_df = pd.read_excel('WFH Curation.xlsx',sheet_name='policy_current_state',usecols=[4,5],skiprows=[0]).dropna()

In [11]:

neg_id=topic_sub_df.loc[topic_sub_df['Topic.1'].isin(['Low Content'])]['Topic.1']
neg_id=pd.concat([neg_id,pd.Series(['Outliers'])])

res_df= pd.DataFrame()

pos_id=topic_sub_df[~ topic_sub_df['Topic.1'].isin(['Low Content'])]['Topic.1'].drop_duplicates()

res_df['title']=pd.concat([neg_id,pos_id]).reset_index(drop=True)
res_df['P_id']=-1000

res_df['labels']=[ k for k in range(-1*len(neg_id),len(pos_id))]
for  topic in pos_id:
    label=res_df['labels'].tail(1).values[0]
    pid=res_df[res_df['title']==topic]['labels'].values[0]
    rdf= topic_sub_df.loc[topic_sub_df['Topic.1']==topic].reset_index(drop=True)
    rdf['P_id']=pid
    rdf['labels']=[i+label+1 for i in rdf.index.values]
    rdf=rdf.drop(columns=['Topic.1'])
    rdf=rdf.rename(columns={"Subtopic.1":"title"})
    #print(rdf)
    res_df=pd.concat([res_df,rdf]).reset_index(drop=True)

res_df.set_index('title',inplace=True)
res_df
topics=res_df[res_df['P_id']==-1000]
subtopics=res_df[res_df['P_id']!=-1000]

In [32]:
topics

,P_id,labels
title,,
Low Content,-1000,-2
Outliers,-1000,-1
Adopted hybrid work policy,-1000,0
Shifted to full remote work,-1000,1
No change in policy,-1000,2
Returned to office,-1000,3


In [40]:
x=pd.merge(
    all_questions_res,
    topics['labels'],
    how="left",
    left_on=['Topic'],
    right_on=[topics.index],
    left_index=False,
    right_index=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

x=pd.merge(
    x,
    subtopics['labels'],
    how="left",
    left_on=['Subtopic'],
    right_on=[subtopics.index],
    left_index=False,
    right_index=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)


In [48]:
y=all_questions_res.join(topics['labels'],on='Topic',how='left')
y=y.rename(columns={'labels':'topic_id'})
y=y[['Response', 'Topic', 'topic_id','Subtopic']]
y=y.join(subtopics['labels'],on='Subtopic',how='inner')
y=y.rename(columns={'labels':'subtopic_id','Response':'Policy_current_state'})

In [49]:
y.set_index('Policy_current_state',inplace=True)

In [50]:
result_df=pd.read_csv('WFH upload - all-topics.csv')

In [52]:
result_df=result_df.join(y,on='policy_current_state',how='inner')

In [53]:
result_df

,leader,location,tenure,policy_current_state,impact_quality,impact_cul_perf,policy_desired_state,Topic,topic_id,Subtopic,subtopic_id
1,No,USA - South,2 years or less,"Yes, it was originally fully on site, then mov...","It might be too early to tell, but the back an...",I joined the company as my team was launched t...,2 days to week as long as not on a pip. If on ...,Adopted hybrid work policy,0,Adopted hybrid work policy,4
2,Yes,USA - Northeast,More than 15 years,we have set aside a few days a week for each t...,No impact. I still need to travel to see people,team relationships take more time and planning...,2-3 days a week in the office. I would spend m...,Adopted hybrid work policy,0,Adopted hybrid work policy,4
3,No,USA - West,11-15 years,return to work; 3 days a week,its great; no changes,no change in particular; everyone likes coming...,3 days a week mandatory working from office,Adopted hybrid work policy,0,Adopted hybrid work policy,4
4,Yes,Canada,More than 15 years,not much change; half employees still work fro...,better less stressful; able to spend more time...,performance improved by 6% team less stressed;...,allow staff to decide so long as it improves p...,No change in policy,2,No change in policy,6
5,Yes,USA - West,More than 15 years,We went to primarily remote for everyone durin...,I like the flexibility for my own personal and...,Performance has been steady or slightly increa...,Ask local employees to be in the office at lea...,Adopted hybrid work policy,0,Adopted hybrid work policy,4
...,...,...,...,...,...,...,...,...,...,...,...
171,No,Europe,2 years or less,"No, it hasn't",NaN,NaN,NaN,No change in policy,2,No change in policy,6
172,Yes,Europe,More than 15 years,"Not really. It's hybrid, but most people show ...",NaN,NaN,NaN,Adopted hybrid work policy,0,Adopted hybrid work policy,4
173,Yes,USA - West,More than 15 years,We now have a hybrid policy; 2 days in office;...,NaN,NaN,NaN,Adopted hybrid work policy,0,Adopted hybrid work policy,4
174,Yes,USA - Northeast,More than 15 years,While we like pineapples more than before the ...,NaN,NaN,NaN,No change in policy,2,No change in policy,6


In [42]:
import pandas as pd
class Analyze_question:
    def __init__(self,wfh_alldata_path,curation_path) -> None:
        self.alldata=pd.read_csv(wfh_alldata_path,skiprows=[1])
        self.curation=pd.ExcelFile(curation_path)
        #self.result=pd.ExcelWriter('WFH_Upload.xlsx',engine='xlsxwriter')
        #self.alldata.to_excel(self.result,sheet_name='all-topics',index=True)
        
        #self.result.save()
       # print(self.alldata)
        for question in self.curation.sheet_names[1:]:
            response_df=self.curation.parse(question,usecols=[0,1,2],skiprows=[0])
            topic_sub_df=self.curation.parse(question,usecols=[4,5],skiprows=[0]).dropna()
            
            insights_df= self.curation.parse(question,usecols=[8,10,11],skiprows=[0]).dropna()
            rep_df=self.rep_texts(topic_sub_df)
            #print(insights_df.columns,rep_df.columns)
            rep_df= rep_df.join(insights_df.rename(columns={'Topic.2':'title'}).set_index('title'),on='title',how='left')

            #print(rep_df)
            self.rep_texts_export(rep_df,question)
            
            rep_df.set_index('title',inplace=True)
            topics=rep_df[rep_df['P_id']==-1000]
            #subtopics=rep_df.sort_values(by=['labels'],ascending=False)
            subtopics=rep_df.reset_index().drop_duplicates(subset='title', keep='last').set_index('title')
            print(subtopics)
            res_df=response_df.join(topics['labels'],on='Topic',how='left')
            res_df=res_df.rename(columns={'labels':question+'_label_0','Topic':question+'_title_0'})
            
            res_df=res_df.join(subtopics['labels'],on='Subtopic',how='left')
            res_df=res_df.rename(columns={'labels':question+'_label_1','Response':question,'Subtopic':question+'title_1'})
            
            res_df[question+'_proximity_score_0']=0.5
            res_df[question+'_proximity_score_1']=0.5
            res_df=res_df[[question,question+'_label_0', question+'_title_0',question+'_proximity_score_0',question+'_label_1', question+'title_1',question+'_proximity_score_1']]
            res_df.set_index(question,inplace=True)
            print(res_df.columns)
            self.alldata=self.alldata.join(res_df,on=question,how='left')
            #print(self.alldata)
            #self.alldata.to_csv('test.csv')
            break
        #self.alldata.to_excel(self.result,sheet_name='all-topics',index=True)
        #self.result.save()
        #self.result.close()
            
        
    def __str__(self) -> str:
        return f'exported successful'
    
    def rep_texts(self,topic_sub_df):
        neg_id=topic_sub_df.loc[topic_sub_df['Topic.1'].isin(['Low Content'])]['Topic.1']
        neg_id=pd.concat([neg_id,pd.Series(['Outliers'])])

        res_df= pd.DataFrame()
        pos_id=topic_sub_df[~ topic_sub_df['Topic.1'].isin(['Low Content'])]['Topic.1'].drop_duplicates()
        res_df['title']=pd.concat([neg_id,pos_id]).reset_index(drop=True)
        res_df['P_id']=-1000
        res_df['labels']=[ k for k in range(-1*len(neg_id),len(pos_id))]
        for  topic in pos_id:
            label=res_df['labels'].tail(1).values[0]
            pid=res_df[res_df['title']==topic]['labels'].values[0]
            rdf= topic_sub_df.loc[topic_sub_df['Topic.1']==topic].reset_index(drop=True)
            rdf['P_id']=pid
            rdf['labels']=[i+label+1 for i in rdf.index.values]
            rdf=rdf.drop(columns=['Topic.1'])
            rdf=rdf.rename(columns={"Subtopic.1":"title"})
            #print(rdf)
            res_df=pd.concat([res_df,rdf]).reset_index(drop=True)
        return res_df
        #res_df.set_index('title',inplace=True)
        #topics=res_df[res_df['P_id']==-1000]
        #subtopics=res_df[res_df['P_id']!=-1000]
    def rep_texts_export(self,res_df,question):
        res_df=res_df.rename(columns={'labels':'label',
                       'P_id':'parent',
                       'Exemplar_1':'exemplar_statement_0',
                       'Exemplar_2':'exemplar_statement_1'})
        res_df['proximity_score']=0.5
        res_df['summary']=None
        res_df['keywords']=None
        res_df['count']=5
        res_df['percentage']=5
        res_df['cohesion_score']=.5
        res_df['exemplar_statement_0_score']=.5
        res_df['exemplar_statement_1_score']=.5

        res_df=res_df[['label','parent',
                    'title',
                    'proximity_score',
                    'summary',
                    'keywords',
                    'count',
                    'percentage',
                    'cohesion_score',
                    'exemplar_statement_0',
                    'exemplar_statement_0_score',
                    'exemplar_statement_1',
                    'exemplar_statement_1_score']]
        res_df.fillna('None',inplace=True)
        #res_df.to_excel(self.result,sheet_name= question+'-rep-texts' ,index=True)
        #self.result.save()
        ##res_df.to_csv('transformed_impact_quality.csv')

obj=Analyze_question('WFH upload - all-topics.csv', 'WFH Curation.xlsx')
#df=pd.read_csv(r'WFH upload - all-topics.csv')
print(obj)

                             P_id  labels  \
title                                       
Low Content                 -1000      -2   
Outliers                    -1000      -1   
Adopted hybrid work policy      0       4   
Shifted to full remote work     1       5   
No change in policy             2       6   
Returned to office              3       7   

                                                                    Exemplar_1  \
title                                                                            
Low Content                                                                NaN   
Outliers                                                                   NaN   
Adopted hybrid work policy   yes, we have much more flexibility about comin...   
Shifted to full remote work  very much so. during the initial shut-downs th...   
No change in policy                    Same policy as before; Exactly the same   
Returned to office           We moved from going zero this to off